In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
import time
from collections import Counter
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#tf.enable_eager_execution()

In [3]:
def create_data(filename):
    
    labels=[]
    feature=[]
    with open(filename,'r') as f:
        line = csv.reader(f)
        next(line,None)
        for row in line:
            feature.append([ np.int32(dictionary[x]) if x in dictionary else 0 for x in row[1].split()])
            labels.append(np.int32(row[0]))
    return feature, labels

In [4]:
def build_vocab(fileName):
    text=[]
    with open(fileName,'r') as f:
        line = csv.reader(f)
        next(line,None)
        for row in line:
            text.extend(row[1].replace('\n','<eos>').split())
    count=[['UNK',1]]
    count.extend(Counter(text).most_common())
    dictionary = dict()
    for word ,_ in count:
        dictionary[word]=len(dictionary)
    reverse_dictionary=dict(zip(dictionary.values(),dictionary.keys()))
    del text
    return dictionary,reverse_dictionary

In [5]:
def padd_row(arr,test=False):
    if test:
        max_len=424
    else:
        max_len=max(len(row) for row in arr)
    arr_padded = np.array([row + [0]*(max_len - len(row)) for row in arr])
    return arr_padded

In [6]:
dictionary,reverse_dictionary=build_vocab('data/train.csv')

In [7]:
len(dictionary)

15694

In [8]:
len(reverse_dictionary)

15694

In [9]:
feature,labels=create_data('data/train.csv')

In [10]:
len(labels)

2724

In [11]:
#feature=np.array(feature)

In [12]:
#feature_padd = padd_row(feature)

In [13]:
#len(feature_padd)

In [14]:
#x_data = tf.convert_to_tensor(feature_padd,dtype=tf.int32,name='feature')

In [15]:
#len(feature_padd)
#x_data[:100,0:20]

In [16]:
def create_batch(feature,label,batch_size,test=False):
    #assert feature.shape[0]%batch_size==0,"Use batch size in multiple of 10"
    feature_padd = padd_row(feature,test)
    #x_data = tf.convert_to_tensor(feature_padd,dtype=tf.int32,name='feature')
    #data_size = tf.size(x_data)
    no_batch = len(feature_padd)//batch_size
    #data = tf.reshape(x_data[:no_batch*batch_size],[batch_size,no_batch]) # or we can user [batch_size,-1]
    #labels = tf.convert_to_tensor(np.array(label),dtype=tf.int32,name='label')
    label=np.array(label)
    x_=feature_padd[:no_batch*batch_size]
    y_=label[:no_batch*batch_size]
    
    for n in range(0, len(x_), batch_size):
        #x = data[:,n:n+seq_n]
        x = x_[n:n+batch_size]
        y = y_[n:n+batch_size]
    
        yield x ,y   
    
    

In [17]:
#batch = create_batch(feature,labels,50,424)
#x,y=next(batch)
#print (x.shape)
#print (y)
#for e in range(20):
    #for x,y in create_batch(feature,labels,100):
        #print (x)
        #print (y)

In [18]:
def create_cell(lstm_size,keep_prob):
    with tf.variable_scope("cells"):
        cell = tf.nn.rnn_cell.LSTMCell(lstm_size)
        drop = tf.nn.rnn_cell.DropoutWrapper(cell,output_keep_prob=keep_prob)
    return drop

In [19]:
def build_input(batch_size,num_setps,num_classes):
    inputs = tf.placeholder(tf.int32,[batch_size,num_setps],name='input')
    targets = tf.placeholder(tf.int32,[batch_size],name='targets')
    keep_proba = tf.placeholder(tf.float32,name='keep_proba')
    
    return inputs,targets,keep_proba

In [20]:
def build_lstm(lstm_size,num_layers,batch_size,keep_proba):
    with tf.name_scope("LSTM_Network"):
        cell = tf.contrib.rnn.MultiRNNCell([create_cell(lstm_size,keep_proba) for _ in range(num_layers)])
        initial_state = cell.zero_state(batch_size,tf.float32)
    
    return cell,initial_state

In [21]:
class YelpNetwork(object):
    def __init__(self,
                 lstm_size,
                 batch_size,
                 num_layers,
                 vocab,
                 numClasses=2,
                 num_steps=424,
                 learning_rate=0.001,
                 grad_clip=0.5,
                 embedding_size=300):
        
        self.lstm_size=lstm_size
        self.batch_size=batch_size
        self.num_layers=num_layers
        self.vocab_size=len(vocab)
        self.numClasses=numClasses
        self.grad_clip=grad_clip
        self.inputs,self.target,self.keep_proba=build_input(self.batch_size,num_steps,numClasses)
        self.learning_rate=learning_rate
        self.embedding= self.embedding_matrix(self.inputs,embedding_size,self.vocab_size)
        self.logits,self.out=self.build_output()
        self.loss=self.build_loss()
        self.optimizer=self.build_optimizer()
        self.accuracy =self.accuracy()
        
    
    def embedding_matrix(self,x,embedding_size,vocab_size):
        with tf.variable_scope('embedding'):
            embedd = tf.Variable(tf.random_uniform([vocab_size,embedding_size],-0.05,0.05))
            embedding=tf.nn.embedding_lookup(embedd,x)
        return embedding
    
    def build_output(self):
        cell,self.initial_state = build_lstm(self.lstm_size,self.num_layers,self.batch_size,self.keep_proba)
        output,state = tf.nn.dynamic_rnn(cell, self.embedding,initial_state=self.initial_state)
        self.final_state = state
        #output_flat = tf.reshape(output,[:,-1])
        
        with tf.variable_scope('softmax',reuse=True):
            softmax_w = tf.Variable(tf.truncated_normal([self.lstm_size,self.numClasses],stddev=0.1))
            softmax_b = tf.Variable(tf.truncated_normal([self.numClasses],stddev=0.1))
        
        logits = tf.nn.xw_plus_b(output[:,-1],softmax_w,softmax_b)
        #logits = tf.sigmoid(logits,name='sigmoid')
        out = tf.nn.sigmoid(logits,name='predictions')
        #out = tf.argmax(out,axis=1)
        return logits,out
    
    def build_loss(self):
        
        with tf.variable_scope("loss"):
            
            y_one_hot = tf.one_hot(self.target,self.numClasses)
            #y_reshaped = tf.reshape(y_one_hot,(self.logits.get_shape()))
        
            loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_one_hot,logits=self.logits)
            loss = tf.reduce_mean(loss)
        
        return loss
    
    def build_optimizer(self):
        
        with tf.variable_scope("optimizer"):
            tvars = tf.trainable_variables()
            grads,_ = tf.clip_by_global_norm(tf.gradients(self.loss,tvars),self.grad_clip)
            train_op = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
            optimizer = train_op.apply_gradients(zip(grads,tvars))
        return optimizer
   
    def accuracy(self):
        with tf.variable_scope("accuracy"):
            y_one_hot_test = tf.one_hot(self.target,self.numClasses)
            correct_predictions = tf.equal(tf.argmax(self.out,1),tf.argmax(y_one_hot_test,1))
            accuracy = tf.reduce_mean(tf.cast(correct_predictions,'float'))
        
        return accuracy
    
       

In [22]:
batch_size = 100         # Sequences per batch
#num_steps = 100          # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
#learning_rate = 0.001    # Learning rate
keep_prob = 0.5   

In [23]:
epoch = 20
save_every_n = 200

model = YelpNetwork(lstm_size,batch_size,num_layers,dictionary)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    
    counter = 0
    
    for e in range (epoch):
        new_state = sess.run(model.initial_state)
        loss = 0.0
        for x,y in create_batch(feature,labels,batch_size):
        #while True:
            #x,y=next(create_batches(encoded, batch_size, num_steps))
            counter +=1
            start = time.time()
            feed = {model.inputs: x,
                    model.target: y,
                    model.keep_proba: keep_prob,
                    model.initial_state: new_state}
            
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer
                                                 ], 
                                                 feed_dict=feed)
            loss +=batch_loss
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epoch),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  #'Training state: {:.4f}... '.format(new_state),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    #print ('Out : ',out) 
    #print ('Logits : ',logits)
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch: 1/20...  Training Step: 1...  Training loss: 0.1450...  21.6550 sec/batch
Epoch: 1/20...  Training Step: 2...  Training loss: 0.1012...  27.7580 sec/batch
Epoch: 1/20...  Training Step: 3...  Training loss: 0.0374...  25.4710 sec/batch
Epoch: 1/20...  Training Step: 4...  Training loss: 0.1000...  23.0020 sec/batch
Epoch: 1/20...  Training Step: 5...  Training loss: 0.2003...  23.5980 sec/batch
Epoch: 1/20...  Training Step: 6...  Training loss: 0.0987...  23.4050 sec/batch
Epoch: 1/20...  Training Step: 7...  Training loss: 0.0950...  22.4700 sec/batch
Epoch: 1/20...  Training Step: 8...  Training loss: 0.1085...  22.0500 sec/batch
Epoch: 1/20...  Training Step: 9...  Training loss: 0.1231...  21.8230 sec/batch
Epoch: 1/

Epoch: 4/20...  Training Step: 93...  Training loss: 0.1869...  29.0770 sec/batch
Epoch: 4/20...  Training Step: 94...  Training loss: 0.1270...  26.2300 sec/batch
Epoch: 4/20...  Training Step: 95...  Training loss: 0.1437...  28.4420 sec/batch
Epoch: 4/20...  Training Step: 96...  Training loss: 0.2196...  28.0050 sec/batch
Epoch: 4/20...  Training Step: 97...  Training loss: 0.1003...  26.3370 sec/batch
Epoch: 4/20...  Training Step: 98...  Training loss: 0.0727...  26.1820 sec/batch
Epoch: 4/20...  Training Step: 99...  Training loss: 0.2920...  27.7360 sec/batch
Epoch: 4/20...  Training Step: 100...  Training loss: 0.1503...  28.2270 sec/batch
Epoch: 4/20...  Training Step: 101...  Training loss: 0.1807...  26.8870 sec/batch
Epoch: 4/20...  Training Step: 102...  Training loss: 0.1056...  27.1570 sec/batch
Epoch: 4/20...  Training Step: 103...  Training loss: 0.1909...  27.1230 sec/batch
Epoch: 4/20...  Training Step: 104...  Training loss: 0.1045...  28.8870 sec/batch
Epoch: 4/20

Epoch: 8/20...  Training Step: 192...  Training loss: 0.1377...  28.6500 sec/batch
Epoch: 8/20...  Training Step: 193...  Training loss: 0.1691...  28.9450 sec/batch
Epoch: 8/20...  Training Step: 194...  Training loss: 0.1256...  29.0190 sec/batch
Epoch: 8/20...  Training Step: 195...  Training loss: 0.1437...  30.8370 sec/batch
Epoch: 8/20...  Training Step: 196...  Training loss: 0.0897...  27.7590 sec/batch
Epoch: 8/20...  Training Step: 197...  Training loss: 0.2266...  29.5720 sec/batch
Epoch: 8/20...  Training Step: 198...  Training loss: 0.1009...  30.5310 sec/batch
Epoch: 8/20...  Training Step: 199...  Training loss: 0.0802...  28.0370 sec/batch
Epoch: 8/20...  Training Step: 200...  Training loss: 0.2574...  28.8110 sec/batch
Epoch: 8/20...  Training Step: 201...  Training loss: 0.0966...  30.8090 sec/batch
Epoch: 8/20...  Training Step: 202...  Training loss: 0.1521...  27.9950 sec/batch
Epoch: 8/20...  Training Step: 203...  Training loss: 0.0880...  30.0180 sec/batch
Epoc

Epoch: 11/20...  Training Step: 291...  Training loss: 0.0981...  29.7360 sec/batch
Epoch: 11/20...  Training Step: 292...  Training loss: 0.1637...  30.3590 sec/batch
Epoch: 11/20...  Training Step: 293...  Training loss: 0.0818...  32.1750 sec/batch
Epoch: 11/20...  Training Step: 294...  Training loss: 0.1506...  28.6240 sec/batch
Epoch: 11/20...  Training Step: 295...  Training loss: 0.3447...  30.1740 sec/batch
Epoch: 11/20...  Training Step: 296...  Training loss: 0.2481...  30.0380 sec/batch
Epoch: 11/20...  Training Step: 297...  Training loss: 0.0920...  29.8530 sec/batch
Epoch: 12/20...  Training Step: 298...  Training loss: 0.3413...  32.2310 sec/batch
Epoch: 12/20...  Training Step: 299...  Training loss: 0.2333...  29.6720 sec/batch
Epoch: 12/20...  Training Step: 300...  Training loss: 0.1586...  31.5230 sec/batch
Epoch: 12/20...  Training Step: 301...  Training loss: 0.0763...  32.9820 sec/batch
Epoch: 12/20...  Training Step: 302...  Training loss: 0.1842...  33.2600 se

Epoch: 15/20...  Training Step: 389...  Training loss: 0.3118...  31.2150 sec/batch
Epoch: 15/20...  Training Step: 390...  Training loss: 0.2105...  31.2360 sec/batch
Epoch: 15/20...  Training Step: 391...  Training loss: 0.0698...  32.9170 sec/batch
Epoch: 15/20...  Training Step: 392...  Training loss: 0.1321...  31.7380 sec/batch
Epoch: 15/20...  Training Step: 393...  Training loss: 0.1943...  28.5750 sec/batch
Epoch: 15/20...  Training Step: 394...  Training loss: 0.0973...  28.8480 sec/batch
Epoch: 15/20...  Training Step: 395...  Training loss: 0.1511...  30.1600 sec/batch
Epoch: 15/20...  Training Step: 396...  Training loss: 0.1281...  31.7720 sec/batch
Epoch: 15/20...  Training Step: 397...  Training loss: 0.1475...  28.8610 sec/batch
Epoch: 15/20...  Training Step: 398...  Training loss: 0.2302...  34.9440 sec/batch
Epoch: 15/20...  Training Step: 399...  Training loss: 0.0503...  33.1860 sec/batch
Epoch: 15/20...  Training Step: 400...  Training loss: 0.1383...  28.7800 se

Epoch: 19/20...  Training Step: 487...  Training loss: 0.2109...  41.1070 sec/batch
Epoch: 19/20...  Training Step: 488...  Training loss: 0.1734...  44.2260 sec/batch
Epoch: 19/20...  Training Step: 489...  Training loss: 0.2016...  38.7360 sec/batch
Epoch: 19/20...  Training Step: 490...  Training loss: 0.1963...  42.7790 sec/batch
Epoch: 19/20...  Training Step: 491...  Training loss: 0.1934...  41.8740 sec/batch
Epoch: 19/20...  Training Step: 492...  Training loss: 0.0906...  42.5860 sec/batch
Epoch: 19/20...  Training Step: 493...  Training loss: 0.1122...  38.9960 sec/batch
Epoch: 19/20...  Training Step: 494...  Training loss: 0.0986...  43.3880 sec/batch
Epoch: 19/20...  Training Step: 495...  Training loss: 0.1047...  44.4160 sec/batch
Epoch: 19/20...  Training Step: 496...  Training loss: 0.1588...  41.1450 sec/batch
Epoch: 19/20...  Training Step: 497...  Training loss: 0.2256...  38.8330 sec/batch
Epoch: 19/20...  Training Step: 498...  Training loss: 0.0801...  38.2160 se

In [23]:
test_feature,test_labels=create_data('data/test.csv')

In [24]:
test_accu=[]
out_put=[]
model = YelpNetwork(lstm_size,batch_size,num_layers,dictionary)
saver=tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess,tf.train.latest_checkpoint('checkpoints'))
    #saver.restore(sess,'checkpoints\\i10_l512.ckpt')
    test_state=sess.run(model.initial_state)
    for x,y in create_batch(test_feature,test_labels,batch_size,test=True):
        feed = {model.inputs: x,
                model.target: y,
                model.keep_proba: 1,
                model.initial_state: test_state}
        accu,test_state,out=sess.run([model.accuracy,model.final_state,model.out],feed_dict=feed)
        test_accu.append(accu)
        out_put.extend(out)
        
    print('Test Accuracy: ',np.mean(test_accu))
        

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

INFO:tensorflow:Restoring parameters from checkpoints\i540_l512.ckpt
Test Accuracy:  0.8038461
